# Regressão Linear Multivariada - Trabalho

## Estudo de caso: Qualidade de Vinhos

Nesta trabalho, treinaremos um modelo de regressão linear usando descendência de gradiente estocástico no conjunto de dados da Qualidade do Vinho. O exemplo pressupõe que uma cópia CSV do conjunto de dados está no diretório de trabalho atual com o nome do arquivo *winequality-white.csv*.

O conjunto de dados de qualidade do vinho envolve a previsão da qualidade dos vinhos brancos em uma escala, com medidas químicas de cada vinho. É um problema de classificação multiclasse, mas também pode ser enquadrado como um problema de regressão. O número de observações para cada classe não é equilibrado. Existem 4.898 observações com 11 variáveis de entrada e 1 variável de saída. Os nomes das variáveis são os seguintes:

1. Fixed acidity.
2. Volatile acidity.
3. Citric acid.
4. Residual sugar.
5. Chlorides.
6. Free sulfur dioxide. 
7. Total sulfur dioxide. 
8. Density.
9. pH.
10. Sulphates.
11. Alcohol.
12. Quality (score between 0 and 10).

O desempenho de referencia de predição do valor médio é um RMSE de aproximadamente 0.148 pontos de qualidade.

Utilize o exemplo apresentado no tutorial e altere-o de forma a carregar os dados e analisar a acurácia de sua solução. 


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from math import sqrt
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing
from random import randrange

In [27]:
import csv
 
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"), delimiter=';')
    dataset = list(lines)[1:]
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [28]:
filename = 'winequality-white.csv'
print(filename)
dataset = loadCsv(filename)
print('Arquivo carregado', filename, 'com', len(dataset), 'linhas')
dataset

winequality-white.csv
Arquivo carregado winequality-white.csv com 4898 linhas


[[7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8, 6.0],
 [6.3, 0.3, 0.34, 1.6, 0.049, 14.0, 132.0, 0.994, 3.3, 0.49, 9.5, 6.0],
 [8.1, 0.28, 0.4, 6.9, 0.05, 30.0, 97.0, 0.9951, 3.26, 0.44, 10.1, 6.0],
 [7.2, 0.23, 0.32, 8.5, 0.058, 47.0, 186.0, 0.9956, 3.19, 0.4, 9.9, 6.0],
 [7.2, 0.23, 0.32, 8.5, 0.058, 47.0, 186.0, 0.9956, 3.19, 0.4, 9.9, 6.0],
 [8.1, 0.28, 0.4, 6.9, 0.05, 30.0, 97.0, 0.9951, 3.26, 0.44, 10.1, 6.0],
 [6.2, 0.32, 0.16, 7.0, 0.045, 30.0, 136.0, 0.9949, 3.18, 0.47, 9.6, 6.0],
 [7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8, 6.0],
 [6.3, 0.3, 0.34, 1.6, 0.049, 14.0, 132.0, 0.994, 3.3, 0.49, 9.5, 6.0],
 [8.1, 0.22, 0.43, 1.5, 0.044, 28.0, 129.0, 0.9938, 3.22, 0.45, 11.0, 6.0],
 [8.1, 0.27, 0.41, 1.45, 0.033, 11.0, 63.0, 0.9908, 2.99, 0.56, 12.0, 5.0],
 [8.6, 0.23, 0.4, 4.2, 0.035, 17.0, 109.0, 0.9947, 3.14, 0.53, 9.7, 5.0],
 [7.9, 0.18, 0.37, 1.2, 0.04, 16.0, 75.0, 0.992, 3.18, 0.63, 10.8, 5.0],
 [6.6, 0.16, 0.4, 1.5, 0.044, 48.0, 143.

In [29]:
# Normalização
min_max_scaler = preprocessing.MinMaxScaler()
dataset_scaled = min_max_scaler.fit_transform(dataset)
data = pd.DataFrame(dataset_scaled)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.307692,0.186275,0.216867,0.308282,0.106825,0.149826,0.373550,0.267785,0.254545,0.267442,0.129032,0.5
1,0.240385,0.215686,0.204819,0.015337,0.118694,0.041812,0.285383,0.132832,0.527273,0.313953,0.241935,0.5
2,0.413462,0.196078,0.240964,0.096626,0.121662,0.097561,0.204176,0.154039,0.490909,0.255814,0.338710,0.5
3,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,0.5
4,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,0.5


In [30]:
dataset = data.values

# Example of making a prediction with coefficients
# Make a prediction
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return yhat

def train_test_split(dataset, splitRatio):
    train = list()
    train_size = splitRatio * len(dataset)
    dataset_copy = list(dataset)
    while len(train) < train_size:
        index = randrange(len(dataset_copy))
        train.append(dataset_copy.pop(index))
    return train, dataset_copy

# dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
coef = [0.5, 0.05, 1.8, 0.01, 0.03, 0.08, 0.09, 0.003, 0.4, 0.1, 0.6, 0.5]

errors = list()

for row in dataset:
    yhat = predict(row, coef)
    error = yhat-row[-1]
    errors.append(yhat-row[-1])
    print("Esperado=%.3f, Predito=%.3f, Erro=%.3f" % (row[-1], yhat, error))
    
    
    
    
    
# Estimate linear regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    print ('Coeficiente Inicial={0}' % (coef))
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            sum_error += error**2
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] - l_rate * error * row[i] 
        print(('epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error)))
    return coef

# Calculate coefficients
# dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]


splitRatio = 0.75
train, test = train_test_split(dataset, splitRatio)

l_rate = 0.001
n_epoch = 10000
coef = coefficients_sgd(train, l_rate, n_epoch)
print(coef)

Esperado=0.500, Predito=1.243, Erro=0.743
Esperado=0.500, Predito=1.332, Erro=0.832
Esperado=0.500, Predito=1.332, Erro=0.832
Esperado=0.500, Predito=1.201, Erro=0.701
Esperado=0.500, Predito=1.201, Erro=0.701
Esperado=0.500, Predito=1.332, Erro=0.832
Esperado=0.500, Predito=1.363, Erro=0.863
Esperado=0.500, Predito=1.243, Erro=0.743
Esperado=0.500, Predito=1.332, Erro=0.832
Esperado=0.500, Predito=1.287, Erro=0.787
Esperado=0.333, Predito=1.481, Erro=1.147
Esperado=0.333, Predito=1.254, Erro=0.920
Esperado=0.333, Predito=1.302, Erro=0.969
Esperado=0.667, Predito=1.351, Erro=0.685
Esperado=0.333, Predito=1.730, Erro=1.397
Esperado=0.667, Predito=1.275, Erro=0.608
Esperado=0.500, Predito=1.554, Erro=1.054
Esperado=0.833, Predito=2.129, Erro=1.296
Esperado=0.500, Predito=1.545, Erro=1.045
Esperado=0.333, Predito=1.369, Erro=1.035
Esperado=0.833, Predito=2.129, Erro=1.296
Esperado=0.667, Predito=1.340, Erro=0.673
Esperado=0.833, Predito=1.354, Erro=0.521
Esperado=0.333, Predito=1.972, Err

Esperado=0.333, Predito=1.504, Erro=1.170
Esperado=0.667, Predito=1.516, Erro=0.850
Esperado=0.333, Predito=1.227, Erro=0.894
Esperado=0.333, Predito=1.170, Erro=0.837
Esperado=0.333, Predito=1.469, Erro=1.135
Esperado=0.500, Predito=1.366, Erro=0.866
Esperado=0.500, Predito=1.269, Erro=0.769
Esperado=0.667, Predito=1.291, Erro=0.624
Esperado=0.500, Predito=1.240, Erro=0.740
Esperado=0.333, Predito=1.564, Erro=1.230
Esperado=0.500, Predito=1.243, Erro=0.743
Esperado=0.500, Predito=1.240, Erro=0.740
Esperado=0.333, Predito=1.564, Erro=1.230
Esperado=0.667, Predito=1.188, Erro=0.522
Esperado=0.500, Predito=1.243, Erro=0.743
Esperado=0.000, Predito=1.633, Erro=1.633
Esperado=0.500, Predito=1.304, Erro=0.804
Esperado=0.500, Predito=1.291, Erro=0.791
Esperado=0.333, Predito=1.249, Erro=0.916
Esperado=0.333, Predito=1.479, Erro=1.146
Esperado=0.500, Predito=1.407, Erro=0.907
Esperado=0.500, Predito=1.314, Erro=0.814
Esperado=0.500, Predito=1.339, Erro=0.839
Esperado=0.500, Predito=1.209, Err

Esperado=0.333, Predito=1.939, Erro=1.606
Esperado=0.667, Predito=1.416, Erro=0.749
Esperado=0.333, Predito=1.146, Erro=0.813
Esperado=0.500, Predito=1.143, Erro=0.643
Esperado=0.667, Predito=1.383, Erro=0.716
Esperado=0.333, Predito=0.946, Erro=0.612
Esperado=0.500, Predito=1.379, Erro=0.879
Esperado=0.500, Predito=1.221, Erro=0.721
Esperado=0.500, Predito=1.089, Erro=0.589
Esperado=0.500, Predito=1.213, Erro=0.713
Esperado=0.500, Predito=1.069, Erro=0.569
Esperado=0.500, Predito=1.083, Erro=0.583
Esperado=0.500, Predito=1.254, Erro=0.754
Esperado=0.500, Predito=1.221, Erro=0.721
Esperado=0.500, Predito=1.297, Erro=0.797
Esperado=0.333, Predito=1.026, Erro=0.693
Esperado=0.667, Predito=1.617, Erro=0.950
Esperado=0.833, Predito=1.424, Erro=0.591
Esperado=0.667, Predito=1.322, Erro=0.655
Esperado=0.333, Predito=1.251, Erro=0.917
Esperado=0.500, Predito=1.525, Erro=1.025
Esperado=0.667, Predito=1.322, Erro=0.655
Esperado=0.333, Predito=1.748, Erro=1.415
Esperado=0.333, Predito=1.525, Err

Esperado=0.333, Predito=1.361, Erro=1.027
Esperado=0.500, Predito=1.040, Erro=0.540
Esperado=0.333, Predito=1.303, Erro=0.970
Esperado=0.333, Predito=1.278, Erro=0.945
Esperado=0.667, Predito=1.601, Erro=0.934
Esperado=0.500, Predito=1.205, Erro=0.705
Esperado=0.333, Predito=1.469, Erro=1.136
Esperado=0.667, Predito=1.117, Erro=0.451
Esperado=0.667, Predito=1.117, Erro=0.451
Esperado=0.667, Predito=1.117, Erro=0.451
Esperado=0.667, Predito=1.117, Erro=0.451
Esperado=0.667, Predito=1.117, Erro=0.451
Esperado=0.667, Predito=1.117, Erro=0.451
Esperado=0.500, Predito=0.951, Erro=0.451
Esperado=0.667, Predito=1.117, Erro=0.451
Esperado=0.167, Predito=1.907, Erro=1.740
Esperado=0.500, Predito=1.073, Erro=0.573
Esperado=0.667, Predito=1.670, Erro=1.003
Esperado=0.333, Predito=1.505, Erro=1.171
Esperado=0.333, Predito=1.216, Erro=0.882
Esperado=0.667, Predito=2.261, Erro=1.595
Esperado=0.333, Predito=1.740, Erro=1.407
Esperado=0.333, Predito=1.433, Erro=1.100
Esperado=0.500, Predito=1.477, Err

Esperado=0.500, Predito=1.360, Erro=0.860
Esperado=0.333, Predito=1.153, Erro=0.820
Esperado=0.500, Predito=1.123, Erro=0.623
Esperado=0.500, Predito=1.219, Erro=0.719
Esperado=0.500, Predito=1.222, Erro=0.722
Esperado=0.500, Predito=1.235, Erro=0.735
Esperado=0.167, Predito=1.408, Erro=1.241
Esperado=0.333, Predito=1.460, Erro=1.126
Esperado=0.333, Predito=1.293, Erro=0.960
Esperado=0.667, Predito=1.196, Erro=0.529
Esperado=0.333, Predito=1.227, Erro=0.893
Esperado=0.333, Predito=1.424, Erro=1.091
Esperado=0.333, Predito=1.447, Erro=1.114
Esperado=0.167, Predito=2.059, Erro=1.893
Esperado=0.500, Predito=1.254, Erro=0.754
Esperado=0.500, Predito=1.254, Erro=0.754
Esperado=0.333, Predito=1.263, Erro=0.929
Esperado=0.500, Predito=1.620, Erro=1.120
Esperado=0.500, Predito=1.254, Erro=0.754
Esperado=0.500, Predito=1.527, Erro=1.027
Esperado=0.500, Predito=1.620, Erro=1.120
Esperado=0.500, Predito=1.527, Erro=1.027
Esperado=0.500, Predito=1.254, Erro=0.754
Esperado=0.500, Predito=1.308, Err

...

epoch=7471, lrate=0.001, error=57.298
epoch=7472, lrate=0.001, error=57.298
epoch=7473, lrate=0.001, error=57.298
epoch=7474, lrate=0.001, error=57.298
epoch=7475, lrate=0.001, error=57.298
epoch=7476, lrate=0.001, error=57.298
epoch=7477, lrate=0.001, error=57.298
epoch=7478, lrate=0.001, error=57.298
epoch=7479, lrate=0.001, error=57.298
epoch=7480, lrate=0.001, error=57.298
epoch=7481, lrate=0.001, error=57.298
epoch=7482, lrate=0.001, error=57.298
epoch=7483, lrate=0.001, error=57.298
epoch=7484, lrate=0.001, error=57.298
epoch=7485, lrate=0.001, error=57.298
epoch=7486, lrate=0.001, error=57.298
epoch=7487, lrate=0.001, error=57.298
epoch=7488, lrate=0.001, error=57.298
epoch=7489, lrate=0.001, error=57.298
epoch=7490, lrate=0.001, error=57.298
epoch=7491, lrate=0.001, error=57.298
epoch=7492, lrate=0.001, error=57.298
epoch=7493, lrate=0.001, error=57.298
epoch=7494, lrate=0.001, error=57.298
epoch=7495, lrate=0.001, error=57.298
epoch=7496, lrate=0.001, error=57.298
epoch=7497, 

epoch=7689, lrate=0.001, error=57.298
epoch=7690, lrate=0.001, error=57.298
epoch=7691, lrate=0.001, error=57.298
epoch=7692, lrate=0.001, error=57.298
epoch=7693, lrate=0.001, error=57.298
epoch=7694, lrate=0.001, error=57.298
epoch=7695, lrate=0.001, error=57.298
epoch=7696, lrate=0.001, error=57.298
epoch=7697, lrate=0.001, error=57.298
epoch=7698, lrate=0.001, error=57.298
epoch=7699, lrate=0.001, error=57.298
epoch=7700, lrate=0.001, error=57.298
epoch=7701, lrate=0.001, error=57.298
epoch=7702, lrate=0.001, error=57.298
epoch=7703, lrate=0.001, error=57.298
epoch=7704, lrate=0.001, error=57.298
epoch=7705, lrate=0.001, error=57.298
epoch=7706, lrate=0.001, error=57.298
epoch=7707, lrate=0.001, error=57.298
epoch=7708, lrate=0.001, error=57.298
epoch=7709, lrate=0.001, error=57.298
epoch=7710, lrate=0.001, error=57.298
epoch=7711, lrate=0.001, error=57.298
epoch=7712, lrate=0.001, error=57.298
epoch=7713, lrate=0.001, error=57.298
epoch=7714, lrate=0.001, error=57.298
epoch=7715, 

epoch=7905, lrate=0.001, error=57.298
epoch=7906, lrate=0.001, error=57.298
epoch=7907, lrate=0.001, error=57.298
epoch=7908, lrate=0.001, error=57.298
epoch=7909, lrate=0.001, error=57.298
epoch=7910, lrate=0.001, error=57.298
epoch=7911, lrate=0.001, error=57.298
epoch=7912, lrate=0.001, error=57.298
epoch=7913, lrate=0.001, error=57.298
epoch=7914, lrate=0.001, error=57.298
epoch=7915, lrate=0.001, error=57.298
epoch=7916, lrate=0.001, error=57.298
epoch=7917, lrate=0.001, error=57.298
epoch=7918, lrate=0.001, error=57.298
epoch=7919, lrate=0.001, error=57.298
epoch=7920, lrate=0.001, error=57.298
epoch=7921, lrate=0.001, error=57.298
epoch=7922, lrate=0.001, error=57.298
epoch=7923, lrate=0.001, error=57.298
epoch=7924, lrate=0.001, error=57.298
epoch=7925, lrate=0.001, error=57.298
epoch=7926, lrate=0.001, error=57.298
epoch=7927, lrate=0.001, error=57.298
epoch=7928, lrate=0.001, error=57.298
epoch=7929, lrate=0.001, error=57.298
epoch=7930, lrate=0.001, error=57.298
epoch=7931, 

epoch=8124, lrate=0.001, error=57.297
epoch=8125, lrate=0.001, error=57.297
epoch=8126, lrate=0.001, error=57.297
epoch=8127, lrate=0.001, error=57.297
epoch=8128, lrate=0.001, error=57.297
epoch=8129, lrate=0.001, error=57.297
epoch=8130, lrate=0.001, error=57.297
epoch=8131, lrate=0.001, error=57.297
epoch=8132, lrate=0.001, error=57.297
epoch=8133, lrate=0.001, error=57.297
epoch=8134, lrate=0.001, error=57.297
epoch=8135, lrate=0.001, error=57.297
epoch=8136, lrate=0.001, error=57.297
epoch=8137, lrate=0.001, error=57.297
epoch=8138, lrate=0.001, error=57.297
epoch=8139, lrate=0.001, error=57.297
epoch=8140, lrate=0.001, error=57.297
epoch=8141, lrate=0.001, error=57.297
epoch=8142, lrate=0.001, error=57.297
epoch=8143, lrate=0.001, error=57.297
epoch=8144, lrate=0.001, error=57.297
epoch=8145, lrate=0.001, error=57.297
epoch=8146, lrate=0.001, error=57.297
epoch=8147, lrate=0.001, error=57.297
epoch=8148, lrate=0.001, error=57.297
epoch=8149, lrate=0.001, error=57.297
epoch=8150, 

KeyboardInterrupt: 